GRU based implementation of a recommendation system. Takes as input 7 movies and their ratings. Predicts the rating given by the user for the 8th movie.

### Imports

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from math import sqrt

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

False
Using device: cpu


In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Dataloaders

In [ ]:
import pandas as pd
import torch
import torch.utils.data as data
from torchvision import transforms
import ast
from torch.nn.utils.rnn import pad_sequence

class MovieDataset(data.Dataset):
    """Movie dataset."""

    def __init__(
        self, ratings_file,test=False
    ):
        """
        Args:
            csv_file (string): Path to the csv file with user,past,future.
        """
        self.ratings_frame = pd.read_csv(
            ratings_file,
            delimiter=",",
            # iterator=True,
        )
        self.test = test

    def __len__(self):
        return len(self.ratings_frame)

    def __getitem__(self, idx):
        data = self.ratings_frame.iloc[idx]
        
        movie_history = eval(data.sequence_movie_ids)
        movie_history_ratings = eval(data.sequence_ratings)
        target_movie_id = movie_history[-1:][0]
        target_movie_rating = movie_history_ratings[-1:][0]
        
        movie_history = torch.LongTensor(movie_history[:-1])
        movie_history_ratings = torch.LongTensor(movie_history_ratings[:-1])
        
        
        
        return movie_history, target_movie_id,  movie_history_ratings, target_movie_rating

In [ ]:
users = pd.read_csv(
    "/content/drive/MyDrive/WSTM_latest/data/users.csv",
    sep=",",
)

ratings = pd.read_csv(
    "/content/drive/MyDrive/WSTM_latest/data/ratings.csv",
    sep=",",
)

movies = pd.read_csv(
    "/content/drive/MyDrive/WSTM_latest/data/movies.csv", sep=","
)

In [ ]:
movies.head(5)

,movie_id,title,genres,year,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,75,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,75,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,75,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,75,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,75,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Model

In [ ]:
class GRURecSys(torch.nn.Module):
    def __init__(self, device, movie_embedding_dim):
        super().__init__()
        self.device = device
        self.movie_embedding_dim = movie_embedding_dim

        #movie embedding layer
        self.embeddings_movie_id = nn.Embedding(
            int(movies.movie_id.max()+1), self.movie_embedding_dim
        )
        self.gru = nn.GRU(input_size= self.movie_embedding_dim, hidden_size=32, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.lin1 = nn.Linear(256, 32)
        self.op = nn.Linear(32,1)
        self.leaky_relu = nn.LeakyReLU()
        

    def forward(self, batch):
      movie_history, target_movie_id,  movie_history_ratings, target_movie_rating = batch
      
      movie_history = movie_history.to(self.device)
      target_movie_id = target_movie_id.to(self.device)
      movie_history_ratings = movie_history_ratings.to(self.device)
      target_movie_rating = target_movie_rating.to(self.device)

      
      movie_history_embedding = self.embeddings_movie_id(movie_history)
      history_ratings = movie_history_ratings.unsqueeze(dim=2)

      movie_history_embedding = torch.mul(movie_history_embedding, history_ratings)
      movie_history_embedding = torch.nn.functional.normalize(movie_history_embedding)


      target_embedding = self.embeddings_movie_id(target_movie_id)
      target_embedding = target_embedding.unsqueeze(dim=1)

      gru_inp = torch.cat([movie_history_embedding, target_embedding], dim=1)
      gru_op, hidden = self.gru(gru_inp)
      gru_op = gru_op.to(self.device)
      hidden = hidden.to(self.device)

      gru_op = gru_op.reshape((gru_op.shape[0], gru_op.shape[1]*gru_op.shape[2]))

      x = self.lin1(gru_op)
      x= self.dropout(x)
      x = self.leaky_relu(x)

      op = self.op(x)

      return op, target_movie_rating




      
        

In [ ]:
train_dataset = MovieDataset("/content/drive/MyDrive/WSTM_latest/data/train.csv")
val_dataset = MovieDataset("/content/drive/MyDrive/WSTM_latest/data/validation.csv")
test_dataset = MovieDataset("/content/drive/MyDrive/WSTM_latest/data/test.csv")

train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=256,
            shuffle=True
        )

val_dataloader = torch.utils.data.DataLoader(
            val_dataset,
            batch_size=512,
            shuffle=True
        )



test_dataloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=512,
            shuffle=True
        )

print("Finished Dataloaders")

Finished Dataloaders


test the model

In [ ]:
m = GRURecSys(device, 64)
m = m.to(device)

In [ ]:
criterion = torch.nn.MSELoss()
for batch in train_dataloader:
  
  out, tgt = m(batch)
  out = out.to(device)
  tgt = tgt.to(device)
  output = out.flatten()
  target = tgt.float().flatten()

  loss = criterion(output, target)
  print(loss)

  break

tensor(14.3828, grad_fn=<MseLossBackward0>)


In [ ]:
from tqdm.notebook import trange, tqdm

def training(model, data_loader, val_dataloader, num_epochs, criterion, optimizer, file_path=None):
  val_loss_lst = []
  train_loss = []
  mae_loss = torch.nn.L1Loss()

  for epoch in trange(num_epochs, desc="training", unit="epoch"):

    with tqdm(data_loader, desc="epoch {}".format(epoch + 1), unit="batch", total=len(data_loader)) as batch_iterator:
        model.train()
        total_loss = 0.0
        running_loss = 0.0
        for i, batch_data in enumerate(batch_iterator, start=1):
            optimizer.zero_grad()
            
            output, target = model(batch_data)
            output = output.flatten()
            target = target.float().flatten()
          
            loss = criterion(output, target)
            total_loss += loss.item()
            running_loss += mae_loss(output, target).item()

            loss.backward()
            optimizer.step()

            batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item(), total_loss=total_loss)

            if(i%200 == 0):
              print(f"Running Train Loss: {running_loss/200}")
              running_loss = 0.0
        
        train_loss.append(total_loss)

        
    print("Validation Set")
    val_loss = validate(model, val_dataloader, criterion)
    val_loss_lst.append(val_loss)

    if file_path is not None:
      torch.save(model.state_dict(), file_path)
  return model


In [ ]:
def validate (model, data_loader, criterion):
  with tqdm(data_loader, unit="batch", total=len(data_loader)) as batch_iterator:
    model.eval()
    val_loss = 0.0
    mae_loss = torch.nn.L1Loss()
    for i, batch_data in enumerate(batch_iterator, start=1):
        
        output, target = model.forward(batch_data)
        output = output.flatten()
        target = target.flatten()

        loss = mae_loss(output, target)
        val_loss += loss.item()
  
        batch_iterator.set_postfix(mean_loss=val_loss / i, current_loss=loss.item(), total_loss = val_loss)

  return val_loss

In [ ]:
MOVIE_EMBEDDING_DIM = 64

gru = GRURecSys(device, MOVIE_EMBEDDING_DIM).to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(gru.parameters(), lr=0.0005)
mae_loss = torch.nn.L1Loss()

training(gru, train_dataloader, test_dataloader, 3, criterion, optimizer)

In [ ]:
torch.save(gru.state_dict(), "gru_wts.pth")

In [ ]:
validate(gru, test_dataloader, criterion)